## Current Python Version

Ensure the python version running in the jupyter kernel is the same as expected

In [14]:
from platform import python_version

print(python_version())


3.7.3


List installed packages, should match the ones in the pipfile (and their dependencies)

In [15]:
pip list

Package              Version 
-------------------- --------
absl-py              0.7.1   
astor                0.8.0   
astroid              2.2.5   
attrs                19.1.0  
backcall             0.1.0   
bleach               3.1.0   
cycler               0.10.0  
dataclasses-json     0.2.14  
decorator            4.4.0   
defusedxml           0.6.0   
entrypoints          0.3     
gast                 0.2.2   
google-pasta         0.1.7   
grpcio               1.22.0  
h5py                 2.9.0   
ipykernel            5.1.1   
ipython              7.7.0   
ipython-genutils     0.2.0   
ipywidgets           7.5.1   
isort                4.3.21  
jedi                 0.14.1  
Jinja2               2.10.1  
jsonschema           3.0.2   
jupyter              1.0.0   
jupyter-client       5.3.1   
jupyter-console      6.0.0   
jupyter-core         4.5.0   
Keras-Applications   1.0.8   
Keras-Preprocessing  1.1.0   
kiwisolver           1.1.0   
lazy-object-proxy    1.4.1   
Markdown  

## Internal Module Imports
Code required to enable project's internally defined modulos into the jupyter notebook environment

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
# from electricity import ElectricalSystem, Load, Generator
from dto import (
    COST_PRESETS, 
    ElectricalSystemSpecs, 
    EpsilonSpecs,
    GeneratorSpecs, 
    LoadSpecs, 
    NodeStatePower, 
    SystemHistory, 
    )
    
from learning import ModelTrainer, ModelTester, LearningParams

/home/fmello/.local/share/virtualenvs/2019-MSc_Thesis-4pUnri4s/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/fmello/.local/share/virtualenvs/2019-MSc_Thesis-4pUnri4s/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/fmello/.local/share/virtualenvs/2019-MSc_Thesis-4pUnri4s/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 

Determine the experiment params

In [5]:
from dto import EpsilonSpecs, ElectricalConstants
import math

trackCostSpecs = ElectricalSystemSpecs(
        loads=[LoadSpecs(id_="L1", basePower=4.0, noiseLevel=0.05)],
        generators=[
            GeneratorSpecs(id_="G1", basePower=1.33, costProfile=COST_PRESETS.COAL, minPower=0.5, maxPower=5.0, noiseLevel=0.05),
            GeneratorSpecs(id_="G2", basePower=1.33, costProfile=COST_PRESETS.OIL, minPower=0.5, maxPower=5.0, noiseLevel=0.05),
            GeneratorSpecs(id_="G3", basePower=1.33, costProfile=COST_PRESETS.GAS, minPower=0.5, maxPower=2.0, noiseLevel=0.05),
        ],
        shouldTrackOptimalCost=True,
    )

speedySpecs = ElectricalSystemSpecs(
        loads=[LoadSpecs(id_="L1", basePower=4.0, noiseLevel=0.05)],
        generators=[
            GeneratorSpecs(id_="G1", basePower=1.33, costProfile=COST_PRESETS.COAL, minPower=0.5, maxPower=5.0, noiseLevel=0.05),
            GeneratorSpecs(id_="G2", basePower=1.33, costProfile=COST_PRESETS.OIL, minPower=0.5, maxPower=5.0, noiseLevel=0.05),
            GeneratorSpecs(id_="G3", basePower=1.33, costProfile=COST_PRESETS.GAS, minPower=0.5, maxPower=2.0, noiseLevel=0.05),
        ],
        shouldTrackOptimalCost=False,
    )

# Droop = +infinity essentially remove the droop component from the power update equation. System reacts as if droop is disabled
# ElectricalConstants().droop = math.inf

Train Agents

In [6]:
from electricity import ElectricalSystemFactory
costElectricalSystem = ElectricalSystemFactory.create(trackCostSpecs)


In [10]:
%%timeit
for i in range(1000):
    costElectricalSystem.updateGenerators([])

13.7 s ± 3.99 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
speedyElectricalSystem = ElectricalSystemFactory.create(trackCostSpecs)

In [13]:
%%timeit
for i in range(1000):
    speedyElectricalSystem.updateGenerators([])

13.9 s ± 2.4 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
